# Assignment 2

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Keywords**: Transformers, Question Answering, CoQA

In [1]:
# IMPORTANT: After executiong this cell, restart the runtime as suggested from the output. It is needed to load the corrrect version of these libraries
!pip install tensorflow==2.6
!pip install tensorflow-gpu==2.6
!pip install tensorflow-addons==0.16.1
!pip install transformers==4.18.0
!pip install Keras==2.6.0

!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 458.4 MB 30 kB/s 
     |████████████████████████████████| 14.9 MB 65.1 MB/s 
     |████████████████████████████████| 132 kB 68.2 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=e6b1409f4e8e6c4eb530eb549c3a6d41451b1fd1eb03710498f3e7470466b524
  Stored in directory: /root/.cache/pip/wheels/f1/60/77/22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=a004ee067a029781ef93ec668f84cf598f6a7e94899d05a1ada30c617b5bd1f0
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=72345 sha256=7244299a4152d99eb2d54cf5ab3644f93e811157f9ccaf3a3408b17dd1be61db
  Stored in directory: /root/.cache/pip/wheels/5f/fd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 458.4 MB 30 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 24.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 22.1 MB/s 
     |████████████████████████████████| 182 kB 78.5 MB/s 
     |████████████████████████████████| 6.6 MB 67.3 MB/s 
     |████████████████████████████████| 880 kB 61.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f5af43d9c756d2c3eac42fb35f99e6405937d53e39d867a47510908e2b563b23
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-py

In [1]:
#Remember to restart the runtime before executing this cell!
import datasets
from functools import partial
import json
import importlib.metadata
import numpy as np
import os
import pandas as pd
import random
import tensorflow as tf
import tensorflow_addons as tfa
from tqdm import tqdm
from transformers import EncoderDecoderModel, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoTokenizer, TFAutoModel, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import urllib.request

In [2]:
# After the runtime restart, check versions
print(f"tensorflow version {importlib.metadata.version('tensorflow')} (expected 2.6)")
print(f"tensorflow-gpu version {importlib.metadata.version('tensorflow-gpu')} (expected 2.6)")
print(f"tensorflow-addons version {importlib.metadata.version('tensorflow_addons')} (expected 0.16.1)")
print(f"transformers version {importlib.metadata.version('transformers')} (expected 4.18.0)")
print(f"keras version {importlib.metadata.version('keras')} (expected 2.6.0)")

tensorflow version 2.6.0 (expected 2.6)
tensorflow-gpu version 2.6.0 (expected 2.6)
tensorflow-addons version 0.16.1 (expected 0.16.1)
transformers version 4.18.0 (expected 4.18.0)
keras version 2.6.0 (expected 2.6.0)


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


## Dataset Download


In [4]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.json')

    if not os.path.exists(data_path):
        print(f"Downloading CoQA {suffix} data split... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        print("Download completed!")

In [5]:
# Train data
train_url = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
download_data(data_path='coqa', url_path=train_url, suffix='train')

# Test data
test_url = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
download_data(data_path='coqa', url_path=test_url, suffix='test')  # <-- Why test? See next slides for an answer!

coqa-train-v1.0.json: 49.0MB [00:08, 5.89MB/s]                            


Download completed!


coqa-dev-v1.0.json: 9.09MB [00:03, 2.93MB/s]                            

Download completed!


## [Task 1] Remove unaswerable QA pairs

- Removing unswerable Q A from train and val sets.


In [6]:
json_test_path='/content/coqa/test.json'
json_train_path='/content/coqa/train.json'
with open(json_train_path) as json_file:
    json_train = json.load(json_file)    
with open(json_test_path) as json_file:
    json_test = json.load(json_file)

In [7]:
print(json_train['data'][0]['story'])

The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. 

The Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. 

In March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. 

The Vatican Secret Archives were separated from the library at the beginning of the 17th

In [8]:
print(json_train['data'][0])#access to first topic
print(json_train['data'][0]['questions'])#access to questions of the first topic
print(json_train['data'][0]['answers'])#access to answers of first topic
print('first question of the first topic: ',json_train['data'][0]['questions'][0])#access to first question
print('answer : ', json_train['data'][0]['answers'][0]) #access to first answer

{'source': 'wikipedia', 'id': '3zotghdk5ibi9cex97fepx7jetpso7', 'filename': 'Vatican_Library.txt', 'story': 'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be 

creating dataframes


In [12]:
def from_json_to_df(json_path):
  df_qa_mixed = pd.read_json(json_path)
  df_qa_mixed.head()
  del df_qa_mixed['version']


  #required columns in our dataframe
  cols = ['id', 'source', 'context', 'question', 'answers' 
          #"span_answer", 
          #"span_answer_start"
          ]
  #list of lists to create our dataframe
  comp_list = []
  for index, row in df_qa_mixed.iterrows():
      for i in range(len(row["data"]["questions"])):
          temp_list = []
          temp_list.append(f'{row["data"]["id"]}_{row["data"]["questions"][i]["turn_id"]}')
          temp_list.append(row["data"]["source"])
          temp_list.append(row["data"]["story"])
          temp_list.append(row["data"]["questions"][i]["input_text"])
          #temp_list.append(row["data"]["answers"][i]["input_text"])   #with this I append only the final answer
          temp_list.append({'text': [row["data"]["answers"][i]["span_text"]],   #with this a dictionary with span_text, span_start and the final answer
                            'answer_start': [row["data"]["answers"][i]["span_start"]],
                            'final_answer': [row["data"]["answers"][i]["input_text"]]
                            })
          #temp_list.append(row["data"]["answers"][i]["span_text"])
          #temp_list.append(row["data"]["answers"][i]["span_start"])
          comp_list.append(temp_list)
  new_df = pd.DataFrame(comp_list, columns=cols) 

  new_df.head()
  return new_df

df_train=from_json_to_df(json_train_path)
df_test=from_json_to_df(json_test_path)

The unanswerable Q&A are the answers that contains 'unknown'.

The row 13th contains unknown in the answer column

In [13]:
df_train[0:20]

,id,source,context,question,answers
0,3zotghdk5ibi9cex97fepx7jetpso7_1,wikipedia,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,"{'text': ['Formally established in 1475'], 'an..."
1,3zotghdk5ibi9cex97fepx7jetpso7_2,wikipedia,"The Vatican Apostolic Library (), more commonl...",what is the library for?,{'text': ['he Vatican Library is a research li...
2,3zotghdk5ibi9cex97fepx7jetpso7_3,wikipedia,"The Vatican Apostolic Library (), more commonl...",for what subjects?,{'text': ['Vatican Library is a research libra...
3,3zotghdk5ibi9cex97fepx7jetpso7_4,wikipedia,"The Vatican Apostolic Library (), more commonl...",and?,{'text': ['Vatican Library is a research libra...
4,3zotghdk5ibi9cex97fepx7jetpso7_5,wikipedia,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,"{'text': ['March 2014, the Vatican Library beg..."
5,3zotghdk5ibi9cex97fepx7jetpso7_6,wikipedia,"The Vatican Apostolic Library (), more commonl...",how do scholars divide the library?,{'text': ['Scholars have traditionally divided...
6,3zotghdk5ibi9cex97fepx7jetpso7_7,wikipedia,"The Vatican Apostolic Library (), more commonl...",how many?,{'text': ['Scholars have traditionally divided...
7,3zotghdk5ibi9cex97fepx7jetpso7_8,wikipedia,"The Vatican Apostolic Library (), more commonl...",what is the official name of the Vat?,"{'text': ['Vatican Apostolic Library (), more ..."
8,3zotghdk5ibi9cex97fepx7jetpso7_9,wikipedia,"The Vatican Apostolic Library (), more commonl...",where is it?,"{'text': ['is the library of the Holy See, loc..."
9,3zotghdk5ibi9cex97fepx7jetpso7_10,wikipedia,"The Vatican Apostolic Library (), more commonl...",how many printed books does it contain?,"{'text': [' It has 75,000 codices from through..."


In [14]:
df_train=df_train.loc[np.in1d(df_train["answers"], [{'text': ['unknown'], 'answer_start': [-1], 'final_answer': ['unknown']}], invert=True)]
df_test=df_test.loc[np.in1d(df_test["answers"], [{'text': ['unknown'], 'answer_start': [-1], 'final_answer': ['unknown']}], invert=True)]

The row 13th does not contain unknown in the answer column

In [15]:
df_train[0:20]

,id,source,context,question,answers
0,3zotghdk5ibi9cex97fepx7jetpso7_1,wikipedia,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,"{'text': ['Formally established in 1475'], 'an..."
1,3zotghdk5ibi9cex97fepx7jetpso7_2,wikipedia,"The Vatican Apostolic Library (), more commonl...",what is the library for?,{'text': ['he Vatican Library is a research li...
2,3zotghdk5ibi9cex97fepx7jetpso7_3,wikipedia,"The Vatican Apostolic Library (), more commonl...",for what subjects?,{'text': ['Vatican Library is a research libra...
3,3zotghdk5ibi9cex97fepx7jetpso7_4,wikipedia,"The Vatican Apostolic Library (), more commonl...",and?,{'text': ['Vatican Library is a research libra...
4,3zotghdk5ibi9cex97fepx7jetpso7_5,wikipedia,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,"{'text': ['March 2014, the Vatican Library beg..."
5,3zotghdk5ibi9cex97fepx7jetpso7_6,wikipedia,"The Vatican Apostolic Library (), more commonl...",how do scholars divide the library?,{'text': ['Scholars have traditionally divided...
6,3zotghdk5ibi9cex97fepx7jetpso7_7,wikipedia,"The Vatican Apostolic Library (), more commonl...",how many?,{'text': ['Scholars have traditionally divided...
7,3zotghdk5ibi9cex97fepx7jetpso7_8,wikipedia,"The Vatican Apostolic Library (), more commonl...",what is the official name of the Vat?,"{'text': ['Vatican Apostolic Library (), more ..."
8,3zotghdk5ibi9cex97fepx7jetpso7_9,wikipedia,"The Vatican Apostolic Library (), more commonl...",where is it?,"{'text': ['is the library of the Holy See, loc..."
9,3zotghdk5ibi9cex97fepx7jetpso7_10,wikipedia,"The Vatican Apostolic Library (), more commonl...",how many printed books does it contain?,"{'text': [' It has 75,000 codices from through..."


Correct the indexes values

In [16]:
df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

In [17]:
df_train[0:20]

,id,source,context,question,answers
0,3zotghdk5ibi9cex97fepx7jetpso7_1,wikipedia,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,"{'text': ['Formally established in 1475'], 'an..."
1,3zotghdk5ibi9cex97fepx7jetpso7_2,wikipedia,"The Vatican Apostolic Library (), more commonl...",what is the library for?,{'text': ['he Vatican Library is a research li...
2,3zotghdk5ibi9cex97fepx7jetpso7_3,wikipedia,"The Vatican Apostolic Library (), more commonl...",for what subjects?,{'text': ['Vatican Library is a research libra...
3,3zotghdk5ibi9cex97fepx7jetpso7_4,wikipedia,"The Vatican Apostolic Library (), more commonl...",and?,{'text': ['Vatican Library is a research libra...
4,3zotghdk5ibi9cex97fepx7jetpso7_5,wikipedia,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,"{'text': ['March 2014, the Vatican Library beg..."
5,3zotghdk5ibi9cex97fepx7jetpso7_6,wikipedia,"The Vatican Apostolic Library (), more commonl...",how do scholars divide the library?,{'text': ['Scholars have traditionally divided...
6,3zotghdk5ibi9cex97fepx7jetpso7_7,wikipedia,"The Vatican Apostolic Library (), more commonl...",how many?,{'text': ['Scholars have traditionally divided...
7,3zotghdk5ibi9cex97fepx7jetpso7_8,wikipedia,"The Vatican Apostolic Library (), more commonl...",what is the official name of the Vat?,"{'text': ['Vatican Apostolic Library (), more ..."
8,3zotghdk5ibi9cex97fepx7jetpso7_9,wikipedia,"The Vatican Apostolic Library (), more commonl...",where is it?,"{'text': ['is the library of the Holy See, loc..."
9,3zotghdk5ibi9cex97fepx7jetpso7_10,wikipedia,"The Vatican Apostolic Library (), more commonl...",how many printed books does it contain?,"{'text': [' It has 75,000 codices from through..."


#### Data Inspection

Spend some time in checking accurately the dataset format and how to retrieve the tasks' inputs and outputs!

## [Task 2] Train, Validation and Test splits

CoQA only provides a train and validation set since the test set is hidden for evaluation purposes.

We'll consider the provided validation set as a test set. <br>
$\rightarrow$ Write your own script to:
* Split the train data in train and validation splits (80% train and 20% val)
* Perform splits such that a dialogue appears in one split only! (i.e., split at dialogue level)
* Perform splitting using the following seed for reproducibility: 42

#### Reproducibility Memo

Check back tutorial 2 on how to fix a specific random seed for reproducibility!

Split data in train and validation

Fix the seed

In [18]:
def set_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
    #os.environ['TF_DISABLE_SEGMENT_REDUCTION_OP_DETERMINISM_EXCEPTIONS'] = '1'

In [19]:
set_reproducibility(42)
unique_text=np.unique(df_train.context)
np.random.shuffle(unique_text)

In [20]:
text_train= unique_text[:int(0.8*len(unique_text))]
text_val=unique_text[int(0.8*len(unique_text)):]

In [21]:
df_val = df_train.loc[np.in1d(df_train['context'], text_val)].reset_index(inplace=False, drop=True)
df_train = df_train.loc[np.in1d(df_train['context'], text_train)].reset_index(inplace=False, drop=True)

In [22]:
# ds_train = datasets.Dataset.from_pandas(df_train)
# ds_val =  datasets.Dataset.from_pandas(df_val)
# ds_test =  datasets.Dataset.from_pandas(df_test)

In [23]:
datasets_dict = datasets.DatasetDict({"train": datasets.Dataset.from_pandas(df_train),
                                      "validation": datasets.Dataset.from_pandas(df_val),
                                      "test": datasets.Dataset.from_pandas(df_test)                
                                      })

## Preprocessing the training data

In [24]:
model_checkpoint = 'prajjwal1/bert-tiny'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [25]:
max_length = 512  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.
pad_on_right = tokenizer.padding_side == "right"

In [26]:
def prepare_train_features(examples, tokenizer, pad_on_right, max_length, doc_stride):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

prepare_features_bert_tiny = partial(prepare_train_features, 
                                     tokenizer = tokenizer, 
                                     pad_on_right=pad_on_right, 
                                     max_length=max_length, 
                                     doc_stride=doc_stride)

In [27]:
tokenized_datasets = datasets_dict.map(
    prepare_features_bert_tiny, batched=True, remove_columns=datasets_dict["train"].column_names
)

  0%|          | 0/86 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

## Fine-tuning the model

In [28]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model

In [32]:
batch_size = 16

In [44]:
model_full_name = "ExtractSpan-bert-tiny-Pieroni-Altieri-Butt"
args = TrainingArguments(
    model_full_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,    #comment this line if you don'want to push the model
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [37]:
from transformers import default_data_collator

data_collator = default_data_collator

In [38]:

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
     

In [39]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 88235
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 16545


Epoch,Training Loss,Validation Loss
1,4.760100,4.547994
2,4.542500,4.344989
3,4.470400,4.304972


Saving model checkpoint to ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-500
Configuration saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-500/config.json
Model weights saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-500/tokenizer_config.json
Special tokens file saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-1000
Configuration saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-1000/config.json
Model weights saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/checkpoint-1000/special_tokens_map.js

TrainOutput(global_step=16545, training_loss=4.73877282740667, metrics={'train_runtime': 683.6915, 'train_samples_per_second': 387.17, 'train_steps_per_second': 24.2, 'total_flos': 322877146798080.0, 'train_loss': 4.73877282740667, 'epoch': 3.0})

In [42]:
trainer.save_model("ExtractSpan-bert-tiny-Pieroni-Altieri-Butt")

Saving model checkpoint to ExtractSpan-bert-tiny-Pieroni-Altieri-Butt
Configuration saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/config.json
Model weights saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/pytorch_model.bin
tokenizer config file saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/tokenizer_config.json
Special tokens file saved in ExtractSpan-bert-tiny-Pieroni-Altieri-Butt/special_tokens_map.json


## Evaluation

In [49]:
import torch
import numpy as np

In [30]:


for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [31]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 512]), torch.Size([16, 512]))

In [32]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 10,   8, 137, 159, 163, 209,  71,  12,   6,  92, 198, 194, 123, 167,
          88,  11], device='cuda:0'),
 tensor([ 46,  11, 144, 159, 165, 215, 100,  29,  36,  95, 201, 217,  11, 165,
          91,  28], device='cuda:0'))

In [47]:
n_best_size = 20

In [48]:


start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

NameError: ignored

In [50]:
def prepare_validation_features(examples, tokenizer, pad_on_right, max_length, doc_stride):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    #examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

prepare_val_features_bert_tiny = partial(prepare_validation_features, 
                                     tokenizer = tokenizer, 
                                     pad_on_right=pad_on_right, 
                                     max_length=max_length, 
                                     doc_stride=doc_stride)

In [51]:
validation_features = datasets_dict["validation"].map(
    prepare_val_features_bert_tiny,
    batched=True,
    remove_columns=datasets_dict["validation"].column_names
)

  0%|          | 0/22 [00:00<?, ?ba/s]

In [52]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 22025
  Batch size = 16


In [53]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [54]:
max_answer_length = 30

In [44]:

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets_dict["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

In [45]:
valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 9.347819,
  'text': 'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat'},
 {'score': 8.739416, 'text': '. Formally established in 1475'},
 {'score': 8.638232, 'text': 'established in 1475'},
 {'score': 8.578568,
  'text': 'the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475'},
 {'score': 8.524745, 'text': '1475'},
 {'score': 8.452339, 'text': 'the Vat'},
 {'score': 8.29668, 'text': 'Formally established in 1475'},
 {'score': 8.248633,
  'text': 'the Vat, is the library of the Holy See, located in Vatican City'},
 {'score': 8.232143, 'text': 'Vatican City. Formally established in 1475'},
 {'score': 8.173708, 'text': 'in 1475'},
 {'score': 8.171995,
  'text': 'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy'},
 {'score': 8.098813,
  'text': 'Vat, is the library of the Holy See, located in Vatican City. Formally establish

In [47]:
datasets_dict["validation"][0]["answers"]

{'answer_start': [151],
 'final_answer': ['It was formally established in 1475'],
 'text': ['Formally established in 1475']}

In [55]:
import collections

examples = datasets_dict["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [56]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [57]:
final_predictions = postprocess_qa_predictions(datasets_dict["validation"], validation_features, raw_predictions.predictions)

Post-processing 21485 example predictions split into 22025 features.


  0%|          | 0/21485 [00:00<?, ?it/s]

In [65]:
from datasets.utils.logging import set_verbosity_error
set_verbosity_error() #disable progress bar

for key in list(final_predictions.keys())[:50]:
  dataset_row = datasets_dict["validation"].filter(lambda x: x["id"] == key)[0]
  print(f'True span: {dataset_row["answers"]["text"]}')
  print(f"Generated span. {final_predictions[key]}")


  0%|          | 0/22 [00:00<?, ?ba/s]

True span: ['Formally established in 1475']
Generated span. The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat


  0%|          | 0/22 [00:00<?, ?ba/s]

True span: ['he Vatican Library is a research library']
Generated span. The Vatican Apostolic Library


  0%|          | 0/22 [00:00<?, ?ba/s]

...

True span: ['beaten to death']
Generated span. Robert Champion


  0%|          | 0/22 [00:00<?, ?ba/s]

True span: ['Bus C']
Generated span. death


  0%|          | 0/22 [00:00<?, ?ba/s]

True span: ['Bus C']
Generated span. According to court documents, Bus C


## [Task 3] Model definition

Write your own script to define the following transformer-based models from [huggingface](https://HuggingFace.co/).

* [M1] DistilRoBERTa (distilberta-base)
* [M2] BERTTiny (bert-tiny)

**Note**: Remember to install the ```transformers``` python package!

**Note**: We consider small transformer models for computational reasons!

M1 DistilRoBERTa (distilberta-base)

## [Task 4] Question generation with text passage $P$ and question $Q$

We want to define $f_\theta(P, Q)$. 

Write your own script to implement $f_\theta$ for each model: M1 and M2.

#### Formulation

Consider a dialogue on text passage $P$. 

For each question $Q_i$ at dialogue turn $i$, your model should take $P$ and $Q_i$ and generate $A_i$.

## [Task 5] Question generation with text passage $P$, question $Q$ and dialogue history $H$

We want to define $f_\theta(P, Q, H)$. Write your own script to implement $f_\theta$ for each model: M1 and M2.

#### Formulation

Consider a dialogue on text passage $P$. 

For each question $Q_i$ at dialogue turn $i$, your model should take $P$, $Q_i$, and $H = \{ Q_0, A_0, \dots, Q_{i-1}, A_{i-1} \}$ to generate $A_i$.

## [Task 6] Train and evaluate $f_\theta(P, Q)$ and $f_\theta(P, Q, H)$

Write your own script to train and evaluate your $f_\theta(P, Q)$ and $f_\theta(P, Q, H)$ models.

### Instructions

* Perform multiple train/evaluation seed runs: [42, 2022, 1337].$^1$
* Evaluate your models with the following metrics: SQUAD F1-score.$^2$
* Fine-tune each transformer-based models for **3 epochs**.
* Report evaluation SQUAD F1-score computed on the validation and test sets.

$^1$ Remember what we said about code reproducibility in Tutorial 2!

$^2$ You can use ```allennlp``` python package for a quick implementation of SQUAD F1-score: ```from allennlp_models.rc.tools import squad```. 

## [Task 7] Error Analysis

Perform a simple and short error analysis as follows:
* Group dialogues by ```source``` and report the worst 5 model errors for each source (w.r.t. SQUAD F1-score).
* Inspect observed results and try to provide some comments (e.g., do the models make errors when faced with a particular question type?)$^1$

$^1$ Check the [paper](https://arxiv.org/pdf/1808.07042.pdf) for some valuable information about question/answer types (e.g., Table 6, Table 8) 

# Assignment Evaluation

The following assignment points will be awarded for each task as follows:

* Task 1, Pre-processing $\rightarrow$ 0.5 points.
* Task 2, Dataset Splitting $\rightarrow$ 0.5 points.
* Task 3 and 4, Models Definition $\rightarrow$ 1.0 points.
* Task 5 and 6, Models Training and Evaluation $\rightarrow$ 2.0 points.
* Task 7, Analysis $\rightarrow$ 1.0 points.
* Report $\rightarrow$ 1.0 points.

**Total** = 6 points <br>

We may award an additional 0.5 points for outstanding submissions. 
 
**Speed Bonus** = 0.5 extra points <br>

# Report

We apply the rules described in Assignment 1 regarding the report.
* Write a clear and concise report following the given overleaf template (**max 2 pages**).
* Report validation and test results in a table.$^1$
* **Avoid reporting** code snippets or copy-paste terminal outputs $\rightarrow$ **Provide a clean schema** of what you want to show

# Comments and Organization

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

Structure your code for readability and maintenance. If you work with Colab, use sections. 

This allows you to build clean and modular code, as well as easy to read and to debug (notebooks can be quite tricky time to time).

# FAQ (READ THIS!)

---

**Question**: Does Task 3 also include data tokenization and conversion step?

**Answer:** Yes! These steps are usually straightforward since ```transformers``` also offers a specific tokenizer for each model.

**Example**: 

```
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
encoded_text = tokenizer(text)
%% Alternatively
inputs = tokenizer.tokenize(text, add_special_tokens=True, max_length=min(max_length, 512))
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
```

**Suggestion**: Hugginface's documentation is full of tutorials and user-friendly APIs.

---
---

**Question**: I'm hitting **out of memory error** when training my models, do you have any suggestions?

**Answer**: Here are some common workarounds:

1. Try decreasing the mini-batch size
2. Try applying a different padding strategy (if you are applying padding): e.g. use quantiles instead of maximum sequence length

---
---

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# The End!

Questions?